In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [2]:
# Cargar el conjunto de datos user_reviews
user_reviews = pd.read_parquet('user_reviews.parquet')

In [6]:
# Mostrar las primeras filas de cada DataFrame
print("Primeras y ultimas filas de user_reviews:")
user_reviews

Primeras y ultimas filas de user_reviews:


,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...
59300,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59301,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59302,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59303,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D


In [7]:
# Obtener información sobre los DataFrames
print("\nInformación de user_reviews:")
print(user_reviews.info())


Información de user_reviews:
<class 'pandas.core.frame.DataFrame'>
Index: 58431 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      58431 non-null  object
 1   user_url     58431 non-null  object
 2   funny        58431 non-null  object
 3   posted       58431 non-null  object
 4   last_edited  58431 non-null  object
 5   item_id      58431 non-null  int64 
 6   helpful      58431 non-null  object
 7   recommend    58431 non-null  bool  
 8   review       58431 non-null  object
dtypes: bool(1), int64(1), object(7)
memory usage: 4.1+ MB
None


In [10]:
# Verificar la forma (número de filas y columnas) del DataFrame
print("\nForma de user_reviews:", user_reviews.shape)


Forma de user_reviews: (58431, 9)


In [11]:
# Verificar valores faltantes en cada DataFrame
print("\nValores faltantes en user_reviews:")
print(user_reviews.isnull().sum())


Valores faltantes en user_reviews:
user_id        0
user_url       0
funny          0
posted         0
last_edited    0
item_id        0
helpful        0
recommend      0
review         0
dtype: int64


In [ ]:
# Crear gráficos de barras para variables categóricas
variables_categoricas = user_reviews.select_dtypes(include=['object']).columns.tolist()
for variable in variables_categoricas:
    plt.figure(figsize=(8, 6))
    user_reviews[variable].value_counts().plot(kind='bar', color='skyblue')
    plt.title(f'Gráfico de Barras de {variable}')
    plt.xlabel(variable)
    plt.ylabel('Frecuencia')
    plt.xticks(rotation=45, ha='right')
    plt.grid(True)
    plt.show()